In [1]:
from sklearn.base import BaseEstimator

In [2]:
class MyDummyClassifier(BaseEstimator):
    def fit(self,x,y=None):
        pass

    def predict(self,x):
        pred = np.zeros((x.shape[0],1))
        for i in range (x.shape[0]):
            # print(pred)
            if x['Sex'].iloc[i] == 1:
                pred[i] = 0
            else:
                pred[i] = 1
                
        return pred

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
def fillna(df):
    df['Age'].fillna(df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    df['Fare'].fillna(0, inplace=True)
    return df

def drop_features(df):
    df.drop(columns=['PassengerId','Name','Ticket'],
            inplace=True)
    return df

def format_features(df):
    from sklearn.preprocessing import LabelEncoder
    df['Cabin'] = df['Cabin'].str[:1]
    features = ['Cabin','Sex','Embarked']
    for feature in features:
        le = LabelEncoder()
        df[feature] = le.fit_transform(df[feature])
    return df

def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = format_features(df)
    return df

In [8]:
df = pd.read_csv('train.csv')
y = df['Survived']
x = df.drop(columns='Survived')
x = transform_features(x)

x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=11)

In [11]:
myclf = MyDummyClassifier()
myclf.fit(x_train,y_train)

pred = myclf.predict(x_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, pred)))

Dummy Classifier의 정확도는: 0.8324


In [13]:
from sklearn.metrics import confusion_matrix

In [15]:
confusion_matrix(y_test, pred)

array([[103,  15],
       [ 15,  46]], dtype=int64)

In [24]:
def get_clf_eval(y_test,pred):
    from sklearn.metrics import accuracy_score, confusion_matrix, precision_score,recall_score
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print(f'정확도:{accuracy:.4f}, 정밀도:{precision:.4f}, 재현율:{recall:.4f}')

In [25]:
get_clf_eval(y_test,pred)

오차행렬
[[104  14]
 [ 13  48]]
정확도:0.8492, 정밀도:0.7742, 재현율:0.7869


In [26]:
from sklearn.linear_model import LogisticRegression

In [27]:
lr_clf = LogisticRegression()
lr_clf.fit(x_train, y_train)
pred = lr_clf.predict(x_test)
get_clf_eval(y_test,pred)

오차행렬
[[104  14]
 [ 13  48]]
정확도:0.8492, 정밀도:0.7742, 재현율:0.7869


C:\Users\admin\AppData\Local\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [29]:
pred_proba = lr_clf.predict_proba(x_test)
np.concatenate([pred_proba,pred.reshape(-1,1)],axis=1)

array([[0.46205513, 0.53794487, 1.        ],
       [0.87870787, 0.12129213, 0.        ],
       [0.8771721 , 0.1228279 , 0.        ],
       [0.88256572, 0.11743428, 0.        ],
       [0.85532099, 0.14467901, 0.        ],
       [0.88220231, 0.11779769, 0.        ],
       [0.88844831, 0.11155169, 0.        ],
       [0.20872256, 0.79127744, 1.        ],
       [0.78289451, 0.21710549, 0.        ],
       [0.3691594 , 0.6308406 , 1.        ],
       [0.89977   , 0.10023   , 0.        ],
       [0.87507533, 0.12492467, 0.        ],
       [0.87716612, 0.12283388, 0.        ],
       [0.88840241, 0.11159759, 0.        ],
       [0.43681649, 0.56318351, 1.        ],
       [0.85901797, 0.14098203, 0.        ],
       [0.90375725, 0.09624275, 0.        ],
       [0.73338708, 0.26661292, 0.        ],
       [0.72483711, 0.27516289, 0.        ],
       [0.17151769, 0.82848231, 1.        ],
       [0.75356422, 0.24643578, 0.        ],
       [0.61903536, 0.38096464, 0.        ],
       [0.

In [30]:
from sklearn.preprocessing import Binarizer

In [32]:
x = [[1,-1,2],
     [2,0,0],
     [0,1.1,1.2]]
binarizer = Binarizer(threshold=1.1) # 1.1보다작으면0 1.1보다크면1
binarizer.fit_transform(x)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [39]:
binarizer = Binarizer(threshold=0.6)
pred_1 = binarizer.fit_transform(pred_proba[:,1].reshape(-1,1))

In [40]:
get_clf_eval(y_test,pred_1)
# 0.5일때
# 오차행렬
# [[104  14]
#  [ 13  48]]
# 정확도:0.8492, 정밀도:0.7742, 재현율:0.7869

# 0.4일때 threshold=0.5에비해 정확도,정밀도떨어지고 재현율오름
# 오차행렬
# [[98 20]
#  [10 51]]
# 정확도:0.8324, 정밀도:0.7183, 재현율:0.8361

# 0.6일때는 threshold=0.4에비해 정확도정밀도 오르고 재현율떨어짐

오차행렬
[[112   6]
 [ 16  45]]
정확도:0.8771, 정밀도:0.8824, 재현율:0.7377
